In [ ]:
import pandas as pd

import numpy as np
#from sklearn.preprocessing import StandardScaler
import pickle


In [ ]:
myDataset = pd.read_csv("Health_Risk_Prediction.csv")
df2 = myDataset.copy() 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# ----------------print("Loaded successfully!\n")------------------------
# 1. Load the saved model and encoders
# ----------------------------------------
load_model = pickle.load(open("DT_Class_FinalModel.sav",'rb'))  #Read Binary from the file  

#with open("encoders.sav", "rb") as f:
 #   encoders = pickle.load(f)
#print("Loaded successfully!\n")

# Columns used for prediction
input_order = ['age', 'sleep','bmi','exercise','sugar_intake','smoking','alcohol']

label_columns = ['exercise','sugar_intake','smoking','alcohol','health_risk']
#label_columns = ['exercise','sugar_intake','smoking','alcohol']  #Removed the output column in input
encoders = {}

for col in label_columns:
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col])   # <--- FIT happens here
    encoders[col] = le

pickle.dump(encoders, open("encoders.sav", "wb"))
print("File SAVED successfully!\n")
print(encoders.keys())

In [ ]:
# ============================
# LOAD MODEL + ENCODERS
# ============================
model = pickle.load(open("model.sav", "rb"))
encoders = pickle.load(open("encoders.sav", "rb"))

print("Loaded model and encoders successfully.")
print("Encoders available:", encoders.keys())


# ============================
# INPUT ORDER (must match training)
# ============================
input_order = [
    "age",
    "bmi",
    "sleep",
    "exercise",
    "sugar_intake",
  #  "profession",
    "smoking",
    "alcohol",
   # "married"
]


# ============================
# ENCODE INPUT VALUES
# ============================
def encode_input(data):
    encoded = []

    for col in input_order:
        value = data[col]

        if col in encoders:
            encoded_val = encoders[col].transform([value])[0]
        else:
            encoded_val = float(value)

        encoded.append(encoded_val)

    return np.array([encoded])


# ============================
# DECODE OUTPUT LABEL (health_risk)
# ============================
def decode_output(pred):
    pred = [int(pred)]  # convert scalar → 1D array
    return encoders["health_risk"].inverse_transform(pred)[0]


# ============================
# OPTIONAL — DECODE INPUT BACK
# ============================
def decode_input(encoded_array):
    decoded = {}

    for i, col in enumerate(input_order):
        value = encoded_array[i]

        if col in encoders:
            decoded[col] = encoders[col].inverse_transform([int(value)])[0]
        else:
            decoded[col] = value

    return decoded


# ============================
# MAIN PREDICTION FLOW
# ============================
def predict_user(data):
    print("\n===== INPUT RECEIVED =====")
    print(data)

    # Encode user input
    encoded_x = encode_input(data)
    print("\nEncoded Input:", encoded_x)

    # Predict
    pred = model.predict(encoded_x)[0]
    print("\nEncoded Prediction:", pred)

    # Decode output
    decoded_result = decode_output(pred)
    print("\nDecoded Health Risk:", decoded_result)

    # Decode input (optional)
    decoded_input = decode_input(encoded_x[0])
    print("\nDecoded Input:", decoded_input)

    print("\n===== FINAL RESULT =====")
    return decoded_result


# ============================
# EXAMPLE USAGE
# ============================
#if __name__ == "__main__":

    # Example user input in string form (same as UI/Frontend)
    user_data = {
        "age": 35,
        "bmi": 26.5,
        "exercise": "yes",
        "sugar_intake": "low",
        "profession": "teacher",
        "smoking": "no",
        "alcohol": "yes",
        "married": "no"
    }




In [ ]:
result = predict_user(user_data)
print("\nPredicted Health Risk:", result)